# Find similar articles with tf-idf + cosine similarity (unsupervised)

In [17]:
import gensim
import numpy as np
import pandas as pd

In [18]:
df_input = pd.read_csv('/home/amoya/.kaggle/competitions/bcpnews/test.csv')

In [19]:
raw_documents = df_input.text.values
print("Number of documents:",len(raw_documents))

Number of documents: 5225


<h6>Example</h6>
We will focus on article 210 that talks about Cepsa

In [20]:
df_input['text'][df_input['text'].str.contains('Cepsa')]

7       La Compañía Nacional de Petróleo de Abu Dabi (...
134     La mayor inversión corresponderá a la planta d...
210     Cepsa ha anunciado hoy la adjudicación de un m...
239     Cepsa, la segunda petrolera en España, conside...
484     Las cifras son elocuentes: de 2001 a 2017 este...
817     El desarrollo del coche eléctrico en España su...
1837    El Ayuntamiento de Madrid cumple su palabra y ...
3408    Los cheques regalo y los vuelos, las últimas o...
3736    Seis de los diez mayores prestamistas en 2017 ...
4292    Los plazos terminan entre el 28 de febrero y e...
4668    Ni Repsol ni Cepsa, las dos mayores petroleras...
5149    El dinero vuelve a mirar a España para inverti...
Name: text, dtype: object

In [21]:
print("\n" + df_input.loc[210]['title'])
print('-'*80)
print(df_input.loc[210]['text'][:800] + " ...")


Abu Dabi potencia Cepsa y le da un megacontrato de 1.200 millones en Emiratos
--------------------------------------------------------------------------------
Cepsa ha anunciado hoy la adjudicación de un megacontrato de exploración y producción de hidrocarburos en Emiratos Árabes valorado en 1.500 millones de dólares (algo más de 1.200 millones de euros).

Esta operación es la mayor en el área de adquisición de derechos de hidrocarburos realizada por Cepsa de una sola vez desde que hace cuatro años se hizo con el grupo Coastal Energy, con activos sobre todo en el sudeste asiático, en Malasia y Tailandia.

Cepsa está controlada por Mubadala, el grupo estatal del emirato de Abu Dabi fruto de la fusión de la antigua IPIC y otros activos.

El nuevo contrato supone un respaldo del Gobierno de Abu Dabi a Cepsa, que se ha convertido en una de sus puntas de lanza empresariales dentro y fuera de Emiratos.

El contrato consiste en que la Compañía Nacional ...


In [22]:
# We will now use NLTK to tokenize
from nltk.tokenize import word_tokenize
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in raw_documents]
# print(gen_docs)

In [23]:
# We will create a dictionary from a list of documents. A dictionary maps every word to a number.
dictionary = gensim.corpora.Dictionary(gen_docs)
print("Number of words in dictionary:",len(dictionary))
for i in range(10):
    print(i, dictionary[i])

Number of words in dictionary: 92125
0 ,
1 .
2 1.300
3 250
4 500
5 a
6 abril
7 administración
8 al
9 asegurado


In [24]:
# Now we will create a corpus. A corpus is a list of bags of words. 
# A bag-of-words representation for a document just lists the number of times each word occurs in the document.
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
# print(corpus)

In [25]:
# Now we create a tf-idf model from the corpus. Note that num_nnz is the number of tokens.
tf_idf = gensim.models.TfidfModel(corpus)
print(tf_idf)
s = 0
for i in corpus:
    s += len(i)
print(s)

TfidfModel(num_docs=5225, num_nnz=1417318)
1417318


In [26]:
# Now we will create a similarity measure object in tf-idf space.
sims = gensim.similarities.Similarity('/home/amoya/coding/bcpnews/2_modelling/similarity/',
                                      tf_idf[corpus],
                                      num_features=len(dictionary))
print(sims)
print(type(sims))

Similarity index with 5225 documents in 0 shards (stored under /home/amoya/coding/bcpnews/2_modelling/similarity/)
<class 'gensim.similarities.docsim.Similarity'>


# Validation

Our validation consists on comparing which articles are similar to `210` and see if those are close to the list of possible candidates - these are articles that contain the word `Cepsa`. 

Therefore, the list of possible candidates is:

* __Candidates__ (contain word `Cepsa`):

```json
[7, 134, 210, 239, 484, 817, 1837, 3408, 3736, 4292, 4668, 5149]
```

* __Similars__ (top 7):

```json
[7, 134, 210, 239, 817, 2732, 4668]
```

In [27]:
# Candidates
df_input['text'][df_input['text'].str.contains('Cepsa')]

7       La Compañía Nacional de Petróleo de Abu Dabi (...
134     La mayor inversión corresponderá a la planta d...
210     Cepsa ha anunciado hoy la adjudicación de un m...
239     Cepsa, la segunda petrolera en España, conside...
484     Las cifras son elocuentes: de 2001 a 2017 este...
817     El desarrollo del coche eléctrico en España su...
1837    El Ayuntamiento de Madrid cumple su palabra y ...
3408    Los cheques regalo y los vuelos, las últimas o...
3736    Seis de los diez mayores prestamistas en 2017 ...
4292    Los plazos terminan entre el 28 de febrero y e...
4668    Ni Repsol ni Cepsa, las dos mayores petroleras...
5149    El dinero vuelve a mirar a España para inverti...
Name: text, dtype: object

In [28]:
# Now create a query document and convert it to tf-idf.
text_sample = raw_documents[210]

In [29]:
query_doc = [w.lower() for w in word_tokenize(text_sample)]
# print(query_doc)
query_doc_bow = dictionary.doc2bow(query_doc)
# print(query_doc_bow)
query_doc_tf_idf = tf_idf[query_doc_bow]
# print(query_doc_tf_idf)

In [30]:
# We show an array of document similarities to query.
# We see that the second document is the most similar with the overlapping of socks and force.
text_sample_simis = sims[query_doc_tf_idf]

In [31]:
df = pd.DataFrame(text_sample_simis).reset_index().rename(columns={'index':'id', 0:'similarity'})
df['id'] = df['id'].apply(lambda x: x+1)
df['text'] = raw_documents
df['title'] = df_input['title'].values
df['keywords'] = df_input['keywords'].values

In [32]:
print("\nSimilarities with text 210\n")

for idx, row in df.sort_values('similarity', ascending=False).head(7).iterrows():
    print("-"*80)
    print("\nText:", idx, "(similarity: %s)" % "{:.2%}".format(row['similarity']))
    print("\nKeywords: " + row['keywords'].replace('[','').replace(']',''))
    print("\n" + row['title'])
    print("\n" + row['text'][:250] + " ...")
    print("")


Similarities with text 210

--------------------------------------------------------------------------------

Text: 210 (similarity: 100.00%)

Keywords: megacontrato, la, potencia, que, le, ha, millones, el, en, abu, una, adnoc, dabi, da, cepsa, y, emiratos

Abu Dabi potencia Cepsa y le da un megacontrato de 1.200 millones en Emiratos

Cepsa ha anunciado hoy la adjudicación de un megacontrato de exploración y producción de hidrocarburos en Emiratos Árabes valorado en 1.500 millones de dólares (algo más de 1.200 millones de euros).

Esta operación es la mayor en el área de adquisici ...

--------------------------------------------------------------------------------

Text: 7 (similarity: 67.08%)

Keywords: y, yacimientos, concede, en, la, que, participación, cepsa, el, dos, mar, abu, para, petróleo, del, adnoc, dabi

Abu Dabi concede a Cepsa el 20% de participación en dos yacimientos en el mar

La Compañía Nacional de Petróleo de Abu Dabi (ADNOC, de sus siglas en inglés) ha firmado ho